# Sprawdzanie dokładności modelu
Plik test_data.csv jest dostępny tylko dla jury. Zawiera dane testowe - jednakowe dla wszystkich drużyn. Jest w takim samym formacie co zbiór, który otrzymaliście wraz z zadaniem, tylko już z czystymi danymi.

In [22]:
import pandas as pd
import numpy as np

In [23]:
test_data = pd.read_csv("test_data.csv")

### Miejsce na twój kod:

In [34]:
from catboost import CatBoostClassifier, Pool
import pandas as pd
import numpy as np

In [35]:
#!pip install catboost

In [36]:
# na chwile
# test_data = pd.read_csv("sample.csv")
dataset = test_data

# setup
K_FOLDS = 3
IMPORTANT_FEATURES = 256

important_features_full = list(pd.read_csv("./sorted_by_importance.csv", index_col=0)["0"])
important_features = important_features_full[:256]
labels = list(dataset.Activity.unique())
contFeatures = list(set(dataset.columns) - {'Activity'})
values = dataset[contFeatures].values
col_mean = np.nanmean(values, axis=0)
inds = np.where(np.isnan(values))
values[inds] = np.take(col_mean, inds[1])
dataset[contFeatures] = values



In [37]:
# test
models = [CatBoostClassifier().load_model("model_{}_weights".format(idx)) for idx in range(K_FOLDS)]

def test(models, dataset):
  labels.sort()
  test_X = dataset[important_features].values
  test_y = dataset['Activity'].values
  predictions = []
  for model in models:
    preds_proba = model.predict_proba(Pool(test_X, test_y))
    predictions.append(preds_proba)
  predictions = np.array(predictions)
  preds = np.argmax(np.mean(predictions, axis=0), axis=1)
  final_test_preds = np.array([labels[pred] for pred in preds])
  return final_test_preds

predicted = test(models, dataset)

### Sprawdzanie dokładności:
Predicted - zmienna przechowująca przewidzianą aktywność. Poniższa linijka powinna zwrócić wartość 0-1.

In [38]:
np.mean(predicted == test_data.Activity)

0.9795918367346939